# **Milestone 4:**

Creating two types of neural network–based sentiment analyzers


*   built-in neural-network based sentiment analyzer
*   self-built neural-network based sentiment analyzer



### **Setting up the environment**

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Installing requirements

In [3]:
%%capture
!pip install --upgrade transformers  # make sure compatible with tokenizers
!wget https://raw.githubusercontent.com/crow-intelligence/growth-hacking-sentiment/master/requirements.txt
!pip install -r requirements.txt

Installing apex

In [4]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Writing setup.sh


Writing setup.sh

In [5]:
%%capture
!sh setup.sh

###**Importing the required modules**

In [6]:
# importing relevant libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from transformers import pipeline, BertTokenizer
from simpletransformers.classification import ClassificationModel

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

###**Loading in the small corpus csv**

In [7]:
# importing and reading csv from drive
df = pd.read_csv('/content/drive/MyDrive/Sentiment Analysis for Marketing/Data/small_corpus.csv')

In [8]:
# viewing a sample of the data
df.head()

,ratings,reviews
0,1,Recently UBISOFT had to settle a huge class-ac...
1,1,"code didn't work, got me a refund."
2,1,"these do not work at all, all i get is static ..."
3,1,well let me start by saying that when i first ...
4,1,"Dont waste your money, you will just end up us..."


In [9]:
# checking for null values
df.isnull().sum()

ratings    0
reviews    4
dtype: int64

In [10]:
# filling null values with emptry string ''
df['reviews'] = df['reviews'].fillna('')

### **Classification with a built-in neural network-based sentiment analyzer**

In [11]:
# building sentiment analysis pipeline
# by default, the Transformers library uses a DistilBERT model that was fine-tuned on the Stanford Sentiment Treebank v2 (SST2) task from the GLUE Dataset
nlp = pipeline("sentiment-analysis")

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

In [12]:
# constructing a BERT tokenizer based on WordPiece
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [13]:
# preprocessing and data preparation
# BERT-like models have a fixed limit in sequence length, which is often 512
# process data (e.g., remove whitespaces, converting to into lowercases)
def preprocessing(text):
  '''
  input:
    text: df-column with unprocessed text
  output:
    text_d: choped and en-/decoded text
  '''
  text_d = []
  for review in df['reviews']:
    review = review[:512]
    encoding = tokenizer.encode(review)
    decoding = tokenizer.decode(encoding, skip_special_tokens=True)
    text_d.append(decoding)
  return text_d

In [14]:
# printing two example outputs
# pipeline outputs a dictionary with two keys: 'label' and 'score'
preprocessed_data = preprocessing(df['reviews'])
for i in range(2):
  print(preprocessed_data[i], '\n')
  print(nlp(preprocessed_data[i]), '\n\n--\n')

recently ubisoft had to settle a huge class - action suit brought against the company for bundling ( the notoriously harmful ) starforce drm with its released games. so what the geniuses at the helm do next? they decide to make the same mistake yet again - by choosing the same drm scheme that made bioshock, mass effect and spore infamous : securom 7. xx with limited activations! mass effect can be found in clearance bins only months after its release ; spore not only undersold miserably but also made history as t 

[{'label': 'NEGATIVE', 'score': 0.9991271495819092}] 

--

code didn't work, got me a refund. 

[{'label': 'NEGATIVE', 'score': 0.9996480345726013}] 

--



In [15]:
df['reviews_pp'] = preprocessed_data
df.head()

,ratings,reviews,reviews_pp
0,1,Recently UBISOFT had to settle a huge class-ac...,recently ubisoft had to settle a huge class - ...
1,1,"code didn't work, got me a refund.","code didn't work, got me a refund."
2,1,"these do not work at all, all i get is static ...","these do not work at all, all i get is static ..."
3,1,well let me start by saying that when i first ...,well let me start by saying that when i first ...
4,1,"Dont waste your money, you will just end up us...","dont waste your money, you will just end up us..."


Converting sentiment scores into sentiment classes

In [16]:
# classifying the reviews as positive, neutral or negative
# since the classifier does not contain the neutral class (only positive and negative), it needs to be created on the basis of the probability score of the label scores
def categorize_review(text):
  s = nlp(text)
  if s[0]['label'] == 'POSITIVE' and s[0]['score'] > 0.85:
      return 1
  elif s[0]['label'] == 'NEGATIVE' and s[0]['score'] > 0.95:
      return -1
  else:  
      return 0
  
# If a review is labeled as positive with a probability score of over 75%, it should be classified as positive
# If a review is labeled as negative with a probability score of over 75%, it should be classified as negative
# Everything else should be classified as neutral

In [17]:
# applying the function and augmenting the dataframe
df['sentiment_cat'] = df['reviews_pp'].apply(categorize_review)

Converting ratings into rating classes

In [18]:
# classifying the ratings as positive, neutral or negative
def categorize_rating(rating):
  '''
  input:
    rating: rating of a certain review
  output:
    integer:
      1: for a positive review
      0: for a neutral review 
      -1: for a negative review
  '''
  if rating == 5:
    return 1 
  elif 2 <= rating <= 4:
    return 0
  else:
    return -1

In [19]:
# applying the function and augmenting the dataframe
df['rating_cat'] = df['ratings'].apply(categorize_rating)
df.head()

,ratings,reviews,reviews_pp,sentiment_cat,rating_cat
0,1,Recently UBISOFT had to settle a huge class-ac...,recently ubisoft had to settle a huge class - ...,-1,-1
1,1,"code didn't work, got me a refund.","code didn't work, got me a refund.",-1,-1
2,1,"these do not work at all, all i get is static ...","these do not work at all, all i get is static ...",-1,-1
3,1,well let me start by saying that when i first ...,well let me start by saying that when i first ...,-1,-1
4,1,"Dont waste your money, you will just end up us...","dont waste your money, you will just end up us...",-1,-1


Evaluating the sentiment analyzer

In [20]:
y_pred = list(df['sentiment_cat'])
y_true = list(df['rating_cat'])

a. Accuracy

In [21]:
# accuracy: amount of reviews that are categorized correctly as positive, negative or neutral
# using scikit-learn metric
accuracy = accuracy_score(y_true, y_pred)*100
print(f'The model got {accuracy.round(2)}% of the predictions right.')

The model got 61.07% of the predictions right.


b. Classification report

In [22]:
# classification report: text report showing the main classification metrics
# using scikit-learn metric
cr = classification_report(y_true, y_pred)
print(cr)

              precision    recall  f1-score   support

          -1       0.59      0.89      0.71      1500
           0       0.43      0.08      0.14      1500
           1       0.67      0.86      0.75      1500

    accuracy                           0.61      4500
   macro avg       0.56      0.61      0.53      4500
weighted avg       0.56      0.61      0.53      4500



### **Classification with a self-built neural network-based sentiment analyzer**

In [29]:
# transforming labels since the loss function won't work for negative labels
def transform_labels(label):
    """encode [-1, 0, 1] to [0, 1, 2]"""
    if label == -1:
        return 0  # negative
    elif label == 0:
        return 1  # neutral
    elif label == 1:
        return 2  # positive
    else:
        raise ValueError('unknown label value')

transformed_labels = df['rating_cat'].apply(transform_labels)
transformed_labels.unique()

array([0, 1, 2])

Splitting into training, validation and test test

In [30]:
# creating a training, validation and test set
X = list(df['reviews'])
y = list(encoded_labels)

X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, stratify=y)

X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, stratify=y_train_full)

train_df = pd.DataFrame({
    'text': X_train,
    'labels': y_train,})

val_df = pd.DataFrame({
    'text': X_val,
    'labels': y_val,})

test_df = pd.DataFrame({
    'text': X_test,
    'labels': y_test,})

print(train_df.shape)
print(val_df.shape)
print(test_df.shape)

(2531, 2)
(844, 2)
(1125, 2)


Model configuration & training process

In [31]:
# creating the model
# using simpletransformers ClassificationModel (built on top of the Transformers library by Hugging Face)
# the ClassificationModel class is used for all text classification tasks except for multi label classification
model = ClassificationModel(
    "distilbert",
    "distilbert-base-uncased",
    use_cuda=True,
    num_labels=3,
    args={
        'use_cuda': True, 
        'max_seq_length': 128,
        'num_train_epochs': 10,
        'output_dir': '/content/drive/MyDrive/Sentiment Analysis for Marketing/outputs/',
        'best_model_dir': '/content/drive/MyDrive/Sentiment Analysis for Marketing//outputs/best_model/',
        'evaluate_during_training': True,
        'num_training_epochs': 20,
        'train_batch_size': 20,
        'eval_batch_size': 20
      })

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [32]:
# training the model
model.train_model(train_df=train_df, eval_df=val_df)

  0%|          | 0/2531 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/127 [00:00<?, ?it/s]

  0%|          | 0/844 [00:00<?, ?it/s]

Running Epoch 1 of 10:   0%|          | 0/127 [00:00<?, ?it/s]

...

Running Epoch 8 of 10:   0%|          | 0/127 [00:00<?, ?it/s]

  0%|          | 0/844 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/127 [00:00<?, ?it/s]

  0%|          | 0/844 [00:00<?, ?it/s]

(1270,
 {'eval_loss': [0.8187382013298744,
   0.8069924282473188,
   0.974675869525865,
   1.283232920391615,
   1.4819503398828728,
   1.7624836979910385,
   1.7129898701989374,
   1.792810367983441,
   1.890520955934081,
   1.843089029539463],
  'global_step': [127, 254, 381, 508, 635, 762, 889, 1016, 1143, 1270],
  'mcc': [0.46277706015564046,
   0.4495763039434509,
   0.45022894513005685,
   0.4408810726388039,
   0.4635245173672817,
   0.4120472984697979,
   0.4799155016416804,
   0.4919371152638494,
   0.46823036119600214,
   0.48333486003765963],
  'train_loss': [0.7175840139389038,
   0.648752748966217,
   0.3928728997707367,
   0.1962171494960785,
   0.009276074357330799,
   0.15121930837631226,
   0.02738451212644577,
   0.016408884897828102,
   0.0011780043132603168,
   0.001493785879574716]})

Evaluating the sentiment analyzer

In [34]:
# evaluating the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df=val_df, acc=accuracy_score)
print(result)

  0%|          | 0/844 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/43 [00:00<?, ?it/s]

{'mcc': 0.48333486003765963, 'acc': 0.6552132701421801, 'eval_loss': 1.843089029539463}


In [37]:
y_pred = list(model_outputs.argmax(axis=1))
y_true = y_val

a. Accuracy

In [38]:
# accuracy: amount of reviews that are categorized correctly as positive, negative or neutral
# using scikit-learn metric
accuracy = accuracy_score(y_true, y_pred)*100
print(f'The model got {accuracy.round(2)}% of the predictions right.')

The model got 65.52% of the predictions right.


b. Classification report

In [39]:
# classification report: text report showing the main classification metrics
# using scikit-learn metric
cr = classification_report(y_true, y_pred)
print(cr)

              precision    recall  f1-score   support

           0       0.71      0.65      0.68       281
           1       0.55      0.60      0.57       282
           2       0.72      0.71      0.71       281

    accuracy                           0.66       844
   macro avg       0.66      0.66      0.66       844
weighted avg       0.66      0.66      0.66       844



### **Finetuning the self-built neural network-based sentiment analyzer**

Model configuration & training process

In [40]:
# finetuning the model
# increasing the sequence_length from 128 --> 512
model_512 = ClassificationModel(
    "distilbert",
    "distilbert-base-uncased",
    use_cuda=True,
    num_labels=3,
    args={
        'use_cuda': True, 
        'max_seq_length': 512,
        'num_train_epochs': 10,
        'output_dir': '/content/drive/MyDrive/Sentiment Analysis for Marketing/outputs_m512/',
        'best_model_dir': '/content/drive/MyDrive/Sentiment Analysis for Marketing//outputs/best_model_m512/',
        'evaluate_during_training': True,
        'num_training_epochs': 20,
        'train_batch_size': 20,
        'eval_batch_size': 20
      })

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [41]:
# training the finetuned model
model_512.train_model(train_df=train_df, eval_df=val_df)

  0%|          | 0/2531 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/127 [00:00<?, ?it/s]

  0%|          | 0/844 [00:00<?, ?it/s]

Running Epoch 1 of 10:   0%|          | 0/127 [00:00<?, ?it/s]

...

Running Epoch 8 of 10:   0%|          | 0/127 [00:00<?, ?it/s]

  0%|          | 0/844 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/127 [00:00<?, ?it/s]

  0%|          | 0/844 [00:00<?, ?it/s]

(1270,
 {'eval_loss': [0.7927087822625803,
   0.785042331662289,
   0.8846903408682624,
   1.248641490243202,
   1.397418200969696,
   1.613371426282927,
   1.720530066379281,
   1.8409901427668194,
   1.84936977264493,
   1.8861425477404927],
  'global_step': [127, 254, 381, 508, 635, 762, 889, 1016, 1143, 1270],
  'mcc': [0.46657591536966897,
   0.4756841078054947,
   0.5134017223791005,
   0.4510083493916854,
   0.4929888734522363,
   0.47249388188498687,
   0.461115535929252,
   0.47013265605294136,
   0.47813622457415766,
   0.47699474883233006],
  'train_loss': [1.0378761291503906,
   0.5310162305831909,
   0.48766037821769714,
   0.1333058923482895,
   0.1244797632098198,
   0.15841470658779144,
   0.003794949734583497,
   0.0017462145769968629,
   0.0012575556756928563,
   0.0015826773596927524]})

Evaluating the finetuned sentiment analyzer

In [42]:
# evaluating the finetuned model
result, model_outputs, wrong_predictions = model_512.eval_model(eval_df=val_df, acc=accuracy_score)
print(result)

  0%|          | 0/844 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/43 [00:00<?, ?it/s]

{'mcc': 0.47699474883233006, 'acc': 0.6504739336492891, 'eval_loss': 1.8861425477404927}


In [43]:
y_pred_512 = list(model_outputs.argmax(axis=1))
y_true = y_val

a. Accuracy

In [44]:
# accuracy: amount of reviews that are categorized correctly as positive, negative or neutral
# using scikit-learn metric
accuracy_512 = accuracy_score(y_true, y_pred_512)*100
print(f'The finetuned model got {accuracy_512.round(2)}% of the predictions right.')

The finetuned model got 65.05% of the predictions right.


b. Classification report

In [45]:
# classification report: text report showing the main classification metrics
# using scikit-learn metric
cr_512 = classification_report(y_true, y_pred_512)
print(cr_512)

              precision    recall  f1-score   support

           0       0.70      0.65      0.68       281
           1       0.52      0.59      0.55       282
           2       0.77      0.71      0.74       281

    accuracy                           0.65       844
   macro avg       0.66      0.65      0.65       844
weighted avg       0.66      0.65      0.65       844

